In [ ]:
from calendar import calendar
import pandas as pd
import json
from pathlib import Path
from sqlalchemy import create_engine
import geopandas as gpd
import timeit
# from sqlalchemy.orm import Session,sessionmaker
# Using SQLAlchemy to connect to the Database

from sqlalchemy import create_engine,MetaData,event
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import secrets as Config

# from .utils.log_helper import *

from secrets import *
engine = create_engine(Config.URI, echo=False)

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

session = Session()
target_schema = "metro_api_dev"
Base = declarative_base(metadata=MetaData(schema=target_schema))

def get_db():
    db = Session()
    try:
        print('Connected to the database')
        yield db
    finally:
        db.close()

list_of_gtfs_static_files = ["calendar_dates","calendar","routes","shapes","stop_times","stops","trips"]

def update_gtfs_static_files():
    process_start = timeit.default_timer()
    for file in list_of_gtfs_static_files:
        print('on '+str(file))
        bus_file_path = "../appdata/gtfs-static/gtfs_bus/" + file + '.txt'
        rail_file_path = "../appdata/gtfs-static/gtfs_rail/" + file + '.txt'
        temp_df_bus = pd.read_csv(bus_file_path)
        temp_df_bus['agency_id'] = 'LACMTA'
        temp_df_rail = pd.read_csv(rail_file_path)
        temp_df_rail['agency_id'] = 'LACMTA_Rail'
        if file == "stops":
            update_stops_seperately(file)

        elif file == "shapes":
            temp_gdf_bus = gpd.GeoDataFrame(temp_df_bus, geometry=gpd.points_from_xy(temp_df_bus.shape_pt_lon, temp_df_bus.shape_pt_lat))   
            temp_gdf_rail = gpd.GeoDataFrame(temp_df_rail, geometry=gpd.points_from_xy(temp_df_rail.shape_pt_lon, temp_df_rail.shape_pt_lat))
            shapes_combined_gdf = gpd.GeoDataFrame(pd.concat([temp_gdf_bus, temp_gdf_rail],ignore_index=True),geometry='geometry')
            shapes_combined_gdf['shape_id_sequence'] = shapes_combined_gdf['shape_id'] +'_' +str(shapes_combined_gdf['shape_pt_sequence'])
            shapes_combined_gdf.crs = {'init': 'epsg:4326'}
            shapes_combined_gdf.to_postgis(file,engine,index=False,if_exists="replace",schema=target_schema)

        else:
            combined_temp_df = pd.concat([temp_df_bus, temp_df_rail])
            combined_temp_df.to_sql(file,engine,index=False,if_exists="replace",schema=target_schema)
        process_end = timeit.default_timer()
        print('Updating took {} seconds'.format(process_end - process_start))


def update_stops_seperately(file):
    bus_file_path = "../appdata/gtfs-static/gtfs_bus/" + file + '.txt'
    temp_df_bus = pd.read_csv(bus_file_path)
    # temp_df_bus['geometry'] = [Point(xy) for xy in zip(temp_df_bus.stop_lon, temp_df_bus.stop_lat)] 
    temp_df_bus['agency_id'] = 'LACMTA'
    temp_gdf_bus_stops = gpd.GeoDataFrame(temp_df_bus,geometry=gpd.points_from_xy(temp_df_bus.stop_lon, temp_df_bus.stop_lat))
    temp_gdf_bus_stops.set_crs(epsg=4326, inplace=True)

    rail_file_path = "../appdata/gtfs-static/gtfs_rail/" + file + '.txt'
    temp_df_rail = pd.read_csv(rail_file_path)
    # temp_df_rail['geometry'] = [Point(xy) for xy in zip(temp_df_rail.stop_lon, temp_df_rail.stop_lat)] 
    temp_df_rail['agency_id'] = 'LACMTA_Rail'
    temp_gdf_bus_stops['stop_id'] = temp_gdf_bus_stops['stop_id'].astype('str')
    temp_gdf_bus_stops['stop_code'] = temp_gdf_bus_stops['stop_code'].astype('str')
    temp_gdf_bus_stops['parent_station'] = temp_gdf_bus_stops['parent_station'].astype('str')
    temp_gdf_bus_stops['tpis_name'] = temp_gdf_bus_stops['tpis_name'].astype('str')

    temp_gdf_rail_stops = gpd.GeoDataFrame(temp_df_rail,geometry=gpd.points_from_xy(temp_df_rail.stop_lon, temp_df_rail.stop_lat))
    temp_gdf_rail_stops.set_crs(epsg=4326, inplace=True)
    temp_gdf_rail_stops['stop_id'] = temp_gdf_rail_stops['stop_id'].astype('str')
    temp_gdf_rail_stops['stop_code'] = temp_gdf_rail_stops['stop_code'].astype('str')
    temp_gdf_rail_stops['parent_station'] = temp_gdf_rail_stops['parent_station'].astype('str')
    temp_gdf_rail_stops['tpis_name'] = temp_gdf_rail_stops['tpis_name'].astype('str')

    temp_gdf_rail_stops.to_postgis("stops",engine,schema=target_schema,if_exists="replace",index=False)
    temp_gdf_bus_stops.to_postgis("stops",engine,schema=target_schema,if_exists="append",index=False)

update_gtfs_static_files()